In [0]:
# Librerias
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import mlflow
import mlflow.sklearn  # O la biblioteca que uses para tu modelo

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from scipy.stats import uniform, randint
from random import random, seed
from sklearn.model_selection import RandomizedSearchCV, train_test_split, StratifiedKFold, RepeatedStratifiedKFold

from xgboost import plot_importance

import time
import warnings 
#warnings.filterwarnings("ignore")

%matplotlib inline
import joblib

In [0]:
pip install catboost

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# # 1. Carga de los datos
# data_top10=pd.read_csv('../../Data/data_top10_country.csv') # to read data from github
# data_top10.head(3)

url = 'https://raw.githubusercontent.com/juankquintana/prediccion_salarios/refs/heads/main/Data/data_top10_country.csv'
data_top10= pd.read_csv(url)

In [0]:
# 2.  Separación Entrenamiento y Prueba
data_top10.columns
data_top10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11193 entries, 0 to 11192
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   experience_level    11193 non-null  object
 1   employment_type     11193 non-null  object
 2   job_title           11193 non-null  object
 3   salary_in_usd       11193 non-null  int64 
 4   employee_residence  11193 non-null  object
 5   remote_ratio        11193 non-null  int64 
 6   company_location    11193 non-null  object
 7   company_size        11193 non-null  object
 8   company_country     11193 non-null  object
 9   employee_country    11193 non-null  object
dtypes: int64(2), object(8)
memory usage: 874.6+ KB


In [0]:
# Preparación de Datos
# Treat 'remote_ratio' as a categorical feature by converting it to 'object' type
data_top10['remote_ratio'] = data_top10['remote_ratio'].astype(str)
# Eliminación columnas redundantes o innecesarias en X_r
columns_to_drop_regression = ['salary_in_usd','employee_residence', 'company_location', 'employment_type', 
                              'company_size','remote_ratio'] # 3 ultimas no son importantes para el modelo

# sample for regression
y_r = data_top10['salary_in_usd']
X_r = data_top10.drop(columns_to_drop_regression, axis=1)
X_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11193 entries, 0 to 11192
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   experience_level  11193 non-null  object
 1   job_title         11193 non-null  object
 2   company_country   11193 non-null  object
 3   employee_country  11193 non-null  object
dtypes: object(4)
memory usage: 349.9+ KB


In [0]:
# Separación Train/Test
Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_r, y_r, test_size=0.2, random_state=77)
cat_features = Xr_train.columns.tolist() 

In [0]:
experiment_name = "/Users/p.luissi@uniandes.edu.co/CatBoostRegresion"  

# Busca el experimento por nombre
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    # Si el experimento no existe, lo creamos
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    # Si ya existe, obtenemos su experiment_id
    experiment_id = experiment.experiment_id
     

In [0]:
# Aquí se ejecuta MLflow sin especificar un nombre o id del experimento.
with mlflow.start_run(experiment_id=experiment_id):
    # Definir los parámetros del modelo
    model1 = CatBoostRegressor(
      depth=4,
      iterations=230,
      l2_leaf_reg=2.8,
      learning_rate=0.08,
      cat_features=cat_features, 
      verbose=0  # Suprime la salida del entrenamiento; ajusta según tus necesidades
      )

    # Entrenar el modelo
    model1.fit(Xr_train, yr_train)
    # Predecir en el conjunto de prueba
    yr_pred1 = model1.predict(Xr_test)
    rmse1 = np.sqrt(mean_squared_error(yr_test, yr_pred1))
    print(f'RMSE1: {rmse1}')

    depth = 4  # Asegúrate de definirlo correctamente
    iterations = 230
    l2_leaf_reg = 2.8
    learning_rate = 0.08
    verbose=0
  
    # Registrar los parámetros
    mlflow.log_param("depth", depth)
    mlflow.log_param("iterations", iterations)
    mlflow.log_param("l2_leaf_reg", l2_leaf_reg)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("verbose", verbose)
    
  
    # Registrar el modelo
    mlflow.catboost.log_model(model1, "CatBoost-model1")
  
    # Registrar la métrica
    mlflow.log_metric("rmse", rmse1)
    print(f'rmse: {rmse1:.2f}')

RMSE1: 56054.07140150193


2024/11/17 20:00:20 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.15.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

2024/11/17 20:00:21 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/11/17 20:00:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run unleashed-squid-605 at: https://community.cloud.databricks.com/ml/experiments/626679721578360/runs/fb0202073f4a47a9aaae32117108d0f8.
2024/11/17 20:00:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://community.cloud.databricks.com/ml/experiments/626679721578360.


rmse: 56054.07


In [0]:
# Aquí se ejecuta MLflow sin especificar un nombre o id del experimento.
with mlflow.start_run(experiment_id=experiment_id):
    # Definir los parámetros del modelo
    model2 = CatBoostRegressor(
      depth=6,
      iterations=443,
      l2_leaf_reg=1.4,
      learning_rate=0.05,
      cat_features=cat_features, 
      verbose=0  # Suprime la salida del entrenamiento; ajusta según tus necesidades
      )

    # Entrenar el modelo
    model2.fit(Xr_train, yr_train)
    # Predecir en el conjunto de prueba
    yr_pred2 = model2.predict(Xr_test)
    rmse2 = np.sqrt(mean_squared_error(yr_test, yr_pred2))
    print(f'RMSE1: {rmse2}')

    depth = 6  # Asegúrate de definirlo correctamente
    iterations = 443
    l2_leaf_reg = 1.4
    learning_rate = 0.04
    verbose=0
  
    # Registrar los parámetros
    mlflow.log_param("depth", depth)
    mlflow.log_param("iterations", iterations)
    mlflow.log_param("l2_leaf_reg", l2_leaf_reg)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("verbose", verbose)
    
  
    # Registrar el modelo
    mlflow.catboost.log_model(model2, "CatBoost-model2")
  
    # Registrar la métrica
    mlflow.log_metric("rmse", rmse2)
    print(f'rmse: {rmse2:.2f}')

RMSE1: 56171.43091226378


2024/11/17 20:00:56 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.15.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

2024/11/17 20:00:58 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/11/17 20:00:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run bright-bear-962 at: https://community.cloud.databricks.com/ml/experiments/626679721578360/runs/2151088b3d2e4a4593137ebd144f1d8b.
2024/11/17 20:00:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://community.cloud.databricks.com/ml/experiments/626679721578360.


rmse: 56171.43


In [0]:
# Aquí se ejecuta MLflow sin especificar un nombre o id del experimento.
with mlflow.start_run(experiment_id=experiment_id):
    # Definir los parámetros del modelo
    model3 = CatBoostRegressor(
      depth=3,
      iterations=150,
      l2_leaf_reg=3,
      learning_rate=0.25,
      cat_features=cat_features, 
      verbose=0  # Suprime la salida del entrenamiento; ajusta según tus necesidades
      )

    # Entrenar el modelo
    model3.fit(Xr_train, yr_train)
    # Predecir en el conjunto de prueba
    yr_pred3 = model3.predict(Xr_test)
    rmse3 = np.sqrt(mean_squared_error(yr_test, yr_pred3))
    print(f'RMSE1: {rmse3}')

    depth = 3  # Asegúrate de definirlo correctamente
    iterations = 150
    l2_leaf_reg = 3
    learning_rate = 0.25
    verbose=0
  
    # Registrar los parámetros
    mlflow.log_param("depth", depth)
    mlflow.log_param("iterations", iterations)
    mlflow.log_param("l2_leaf_reg", l2_leaf_reg)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("verbose", verbose)
    
  
    # Registrar el modelo
    mlflow.catboost.log_model(model3, "CatBoost-model2")
  
    # Registrar la métrica
    mlflow.log_metric("rmse", rmse3)
    print(f'rmse: {rmse3:.2f}')

RMSE1: 56105.253999338245


2024/11/17 20:01:37 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.15.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

2024/11/17 20:01:38 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/11/17 20:01:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run awesome-pig-683 at: https://community.cloud.databricks.com/ml/experiments/626679721578360/runs/ae927c27175d460c86d404a282871e27.
2024/11/17 20:01:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://community.cloud.databricks.com/ml/experiments/626679721578360.


rmse: 56105.25


In [0]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error


# Inicializar el modelo con los parámetros dados
model1 = CatBoostRegressor(
    depth=4,
    iterations=230,
    l2_leaf_reg=2.8,
    learning_rate=0.08,
    cat_features=cat_features, 
    verbose=0  # Suprime la salida del entrenamiento; ajusta según tus necesidades
)

# Entrenar el modelo
model1.fit(Xr_train, yr_train)

# Predecir en el conjunto de prueba
yr_pred1 = model1.predict(Xr_test)



In [0]:
rmse1 = np.sqrt(mean_squared_error(yr_test, yr_pred1))
print(f'RMSE1: {rmse1}')

RMSE1: 56054.07140150193


In [0]:
# Inicializar el modelo con los parámetros dados
model2 = CatBoostRegressor(
    depth=6,
    iterations=443,
    l2_leaf_reg=1.4,
    learning_rate=0.05,
    cat_features=cat_features, 

    verbose=0  # Suprime la salida del entrenamiento; ajusta según tus necesidades
)

# Entrenar el modelo
model2.fit(Xr_train, yr_train)

# Predecir en el conjunto de prueba
yr_pred2 = model2.predict(Xr_test)


In [0]:
rmse2 = np.sqrt(mean_squared_error(yr_test, yr_pred2))
print(f'RMSE_2: {rmse2}')

RMSE_2: 56171.43091226378


In [0]:
# Inicializar el modelo con los parámetros dados
model3 = CatBoostRegressor(
    depth=3,
    iterations=150,
    l2_leaf_reg=3,
    learning_rate=0.25,
    cat_features=cat_features, 
    verbose=0  # Suprime la salida del entrenamiento; ajusta según tus necesidades
)

# Entrenar el modelo
model3.fit(Xr_train, yr_train)

# Predecir en el conjunto de prueba
yr_pred3 = model3.predict(Xr_test)

In [0]:
rmse3 = np.sqrt(mean_squared_error(yr_test, yr_pred3))
print(f'RMSE_3: {rmse3}')

RMSE_3: 56105.253999338245


In [0]:

#Save the best model
#joblib.dump(best_cbr_model, 'best_cbr_reg_model.pkl')  